In [ ]:
import planetary_computer
import xarray as xr
import fsspec
import pystac_client

catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1/")

# load in NEX-GDDP data


In [ ]:
collection = catalog.get_collection("nasa-nex-gddp-cmip6")

In [ ]:
collection.summaries.get_list("cmip6:model")[:5]

In [ ]:
collection.summaries.get_list("cmip6:scenario")

In [ ]:
collection.summaries.get_list("cmip6:variable")

In [ ]:
search = catalog.search(
    collections=["nasa-nex-gddp-cmip6"],
    datetime="1950/2000",
    query={"cmip6:model": {"eq": "ACCESS-CM2"}},
)
items = search.get_all_items()
len(items)

In [ ]:
item = items[0]
item.assets

In [ ]:
signed_item = planetary_computer.sign(item)

tasmax = xr.open_dataset(fsspec.open(signed_item.assets["tasmax"].href).open())
tasmax

In [ ]:
gddp = xr.open_mfdataset([fsspec.open(asset.href).open() for asset in signed_item.assets.values()])

In [ ]:
import warnings
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import pandas as pd

warnings.filterwarnings("ignore", message="__len__")
warnings.filterwarnings("ignore", message="Iteration")


fig, axes = plt.subplots(
    figsize=(16, 9),
    ncols=3,
    nrows=3,
    subplot_kw=dict(projection=ccrs.Robinson()),
    sharex=True,
    sharey=True,
)

day = ds.isel(time=0)

for i, (v, data) in enumerate(day.data_vars.items()):
    ax = axes.ravel()[i]
    r = data.plot(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False)
    ax.set(title=v)

fig.suptitle(pd.to_datetime(day.time.data).strftime("%Y-%m-%d"))
plt.tight_layout()

In [ ]:
gddp.lat

# Load in our data


In [ ]:
# Pass as a parameter to be used by papermill: the runid which will be used to create the template for the raw and summary files
run_id = "MIROC6_ssp370_1991_1995_2071_2075_tasmax_21_day_3_year_rolling"
var = "tasmax"
gcm = "MIROC6"
scenario = "ssp370"
ds = xr.open_zarr(f"az://flow-outputs/intermediate/epoch_replaced_gcm/{run_id}.zarr").drop(
    "dayofyear"
)

In [ ]:
ds.lat

In [ ]:
# Same as for obs/trained comparisons in analysis notebooks but do it for comparing with bcsd dataset